In [ ]:
!pip install openai==1.30.0

In [1]:
import openai
import os
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI, AzureOpenAI
current_path = os.getcwd()
main_path = os.path.dirname(current_path)

In [2]:
API_TYPE = "azure"
API_BASE = "https://llmdemo.openai.azure.com/"
API_VERSION = "2023-09-15-preview"
API_KEY = "7ae7bffcd56d4ee89f742dcf20b67269"
DEPLOYMENT_NAME = "gpt-35-turbo"
MODEL_NAME = "gpt-3.5-turbo"

openai.api_type = API_TYPE
openai.api_base = API_BASE
openai.api_version = API_VERSION
openai.api_key = API_KEY

In [3]:
def read_details_file():
    # Construct the full path to the 'details.txt' file
    file_path = f"{main_path}/details.txt"
    
    # Initialize variables
    url = None
    report = None

    # Open and read the file
    with open(file_path, 'r') as file:
        # Read the first two lines
        url = file.readline().strip()
        report = file.readline().strip()
    
    return url, report

url, report_name = read_details_file()
report_name = report_name

In [4]:
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [5]:
def trigger_csv_agent(question):
    current_path = os.getcwd()
    main_path = os.path.dirname(current_path)
    
    csv_list = remove_unnamed_column_from_folder(os.path.join(main_path,report_name,"visual_data_folder"))
    llm = AzureOpenAI(
        openai_api_key=API_KEY,
        deployment_name=DEPLOYMENT_NAME,
        model_name=MODEL_NAME,
        azure_endpoint = API_BASE,
        api_version=API_VERSION,
        temperature=0
    )
    
    agent = create_csv_agent(llm, 
                         csv_list,
                         verbose=True,
                        )
    
    
    
    try:
        check = agent.invoke(question)
        with open(os.path.join(main_path,"csv_agent_response.txt"),'w') as csv_agent_file:
            csv_agent_file.write(check)
    except Exception as e:
        print(e)
        with open(os.path.join(main_path,"csv_agent_response.txt"),'w') as csv_agent_file:
            csv_agent_file.write(str(e))

In [6]:
def remove_unnamed_column_from_folder(folder_path):
    csv_list = []
    try:
        # Iterate through all files in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(folder_path, file_name)
                
                # Read the CSV file
                df = pd.read_csv(file_path)
                
                # Check if 'Unnamed: 0' column exists and drop it
                if 'Unnamed: 0' in df.columns:
                    df.drop(columns=['Unnamed: 0'], inplace=True)
                
                # Save the modified DataFrame back to the CSV file
                df.to_csv(file_path, index=False)
                
                # Add the file name to the list
                csv_list.append(file_path)
        
        print("Successfully processed all CSV files and removed 'Unnamed: 0' columns.")
        print(csv_list)
        return csv_list
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return []


In [7]:
def extract_final_answer(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()

        # Find the start position of 'Final Answer:'
        final_answer_start = content.find('Final Answer:')
        
        if final_answer_start == -1:
            print("Final Answer not found.")
            return

        # Isolate the part of the content starting from 'Final Answer:'
        final_answer_content = content[final_answer_start + len('Final Answer:'):]

        # Find the next occurrences of 'Question:', 'Thought:', 'Action:', 'Action Input:'
        end_markers = ['Question:', 'Thought:', 'Action:', 'Action Input:']
        end_positions = [final_answer_content.find(marker) for marker in end_markers]
        end_positions = [pos for pos in end_positions if pos != -1]

        # Determine the position of the first end marker
        if end_positions:
            end_position = min(end_positions)
            final_answer = final_answer_content[:end_position].strip()
        else:
            final_answer = final_answer_content.strip()

        print(final_answer)

    except Exception as e:
        print(f"An error occurred: {e}")



In [11]:
question = read_txt_file(os.path.join(main_path,"csv_agent_question.txt"))
trigger_csv_agent(question)

Successfully processed all CSV files and removed 'Unnamed: 0' columns.
['C:\\Users\\NITINS\\OneDrive - Capgemini\\CAPGEMINI\\PROJECT\\GEN AI\\powerbi-usecase\\summarization\\bulk-file-handling\\report-narration\\artificial\\visual_data_folder\\artificial_Anomaly Detection_Date slicer.csv', 'C:\\Users\\NITINS\\OneDrive - Capgemini\\CAPGEMINI\\PROJECT\\GEN AI\\powerbi-usecase\\summarization\\bulk-file-handling\\report-narration\\artificial\\visual_data_folder\\artificial_Anomaly Detection_Revenue Won by Manager and Product category.csv', 'C:\\Users\\NITINS\\OneDrive - Capgemini\\CAPGEMINI\\PROJECT\\GEN AI\\powerbi-usecase\\summarization\\bulk-file-handling\\report-narration\\artificial\\visual_data_folder\\artificial_Anomaly Detection_Revenue Won by Product.csv', 'C:\\Users\\NITINS\\OneDrive - Capgemini\\CAPGEMINI\\PROJECT\\GEN AI\\powerbi-usecase\\summarization\\bulk-file-handling\\report-narration\\artificial\\visual_data_folder\\artificial_Anomaly Detection_Revenue Won by Year and Mon